In [33]:
total_value=int(input("Enter the total money(in Numerical form)"))


Enter the total money(in Numerical form)100000000


In [34]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/core/s-and-p-500-companies/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        dataset = pd.read_csv(resource.descriptor['path'])
    

In [35]:
dataset

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [36]:
sym_name=dict()
for i in range(len(dataset["Symbol"])):
    sym_name[dataset["Symbol"][i]]=dataset["Name"][i]
sym_name

{'MMM': '3M',
 'AOS': 'A. O. Smith',
 'ABT': 'Abbott Laboratories',
 'ABBV': 'AbbVie',
 'ABMD': 'Abiomed',
 'ACN': 'Accenture',
 'ATVI': 'Activision Blizzard',
 'ADM': 'ADM',
 'ADBE': 'Adobe',
 'AAP': 'Advance Auto Parts',
 'AMD': 'Advanced Micro Devices',
 'AES': 'AES Corp',
 'AFL': 'Aflac',
 'A': 'Agilent Technologies',
 'APD': 'Air Products & Chemicals',
 'AKAM': 'Akamai Technologies',
 'ALK': 'Alaska Air Group',
 'ALB': 'Albemarle Corporation',
 'ARE': 'Alexandria Real Estate Equities',
 'ALGN': 'Align Technology',
 'ALLE': 'Allegion',
 'LNT': 'Alliant Energy',
 'ALL': 'Allstate Corp',
 'GOOGL': 'Alphabet (Class A)',
 'GOOG': 'Alphabet (Class C)',
 'MO': 'Altria Group',
 'AMZN': 'Amazon',
 'AMCR': 'Amcor',
 'AEE': 'Ameren Corp',
 'AAL': 'American Airlines Group',
 'AEP': 'American Electric Power',
 'AXP': 'American Express',
 'AIG': 'American International Group',
 'AMT': 'American Tower',
 'AWK': 'American Water Works',
 'AMP': 'Ameriprise Financial',
 'ABC': 'AmerisourceBergen',


In [37]:
def splitter(symbols,size):
    li=list()
    start=0
    while start<len(symbols):
        x=""
        if start+size<len(symbols):
            end=start+size
        else:
            end=len(symbols)
        for i in range(start,end):
            x+=(symbols[i]+",")
        x=x[:-1]
        li.append(x)
        start+=size
    return li

In [46]:
import numpy as np
import requests
import time 
#please enter your generated token key in token platform 
token=""
symbols=dataset["Symbol"]
splits=splitter(symbols,80)
li=list()
for sym in splits:
    time.sleep(2)
    #in case you are using actual version instead of sandbox please remove sandbox from url
    url='https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols='+sym+'&token='+token
    data=requests.get(url).json()
    for i in data:
        name=sym_name[i]
        latestprice=data[i]['quote']['latestPrice']
        marketcap=data[i]['quote']['marketCap']
        li.append([name,i,latestprice,marketcap,"N/A"])
stockset=pd.DataFrame(li,columns=["Company","Symbol","Latest Price","Market Capital","Quantity"])
percompanyvalue=total_value/len(stockset["Symbol"])
    

In [47]:
stockset

,Company,Symbol,Latest Price,Market Capital,Quantity
0,3M,MMM,169.60,98825329682,N/A
1,A. O. Smith,AOS,78.30,12556021354,N/A
2,Abbott Laboratories,ABT,128.84,227061240282,N/A
3,AbbVie,ABBV,144.12,244299071875,N/A
4,Abiomed,ABMD,282.34,13181791562,N/A
...,...,...,...,...,...
499,Yum! Brands,YUM,124.14,37401715431,N/A
500,Zebra Technologies,ZBRA,516.30,27059509083,N/A
501,Zimmer Biomet,ZBH,123.09,26448687383,N/A
502,Zions Bancorp,ZION,69.84,10830647936,N/A


In [48]:
for i in range(len(stockset["Symbol"])):
    stockset["Quantity"][i]=(percompanyvalue//stockset["Latest Price"][i])
stockset

<ipython-input-48-d28167c53a5a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stockset["Quantity"][i]=(percompanyvalue//stockset["Latest Price"][i])


,Company,Symbol,Latest Price,Market Capital,Quantity
0,3M,MMM,169.60,98825329682,1169
1,A. O. Smith,AOS,78.30,12556021354,2534
2,Abbott Laboratories,ABT,128.84,227061240282,1539
3,AbbVie,ABBV,144.12,244299071875,1376
4,Abiomed,ABMD,282.34,13181791562,702
...,...,...,...,...,...
499,Yum! Brands,YUM,124.14,37401715431,1598
500,Zebra Technologies,ZBRA,516.30,27059509083,384
501,Zimmer Biomet,ZBH,123.09,26448687383,1611
502,Zions Bancorp,ZION,69.84,10830647936,2840


In [49]:
import xlsxwriter
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
stockset.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [50]:
background_color = '#ffffff'
font_color = '#000000'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [51]:
column_formats = {  'A': ["Company Name",string_format],
                    'B': ['Ticker', string_format],
                    'C': ['Price', dollar_format],
                    'D': ['Market Capitalization', dollar_format],
                    'E': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [52]:
writer.save()